For this demonstration, we'll look at a script that: 
1) steps through a list of geospatial data portals
2) takes a snapshot of DCAT metadata for the items available in each portal, and 
3) compares it to a previous snapshot of DCAT metadata from the same portals to see what has changed

The Big Ten Academic Alliance Geoportal project uses a more complex version of this script to keep track of changes to the publicly available data that are included in its database.  Open geospatial data can be remarkably unstable.  Data providers often seek to have the most up-to-date version of layers, meaning that historical spatial data may not continue to be hosted.  To keep our discovery tool from returning lots of broken results and to capture newly added resources, we frequently re-check data portals. 

There are several different platform technology on which open geospatial data can be found. These platforms have slightly different metadata schemas and different levels of consistency in how each element is formatted.  For this script, we will look at ArcGIS Open Data portals.

As always, we need to start by importing the modules we'll need. Everything here is part of Python's standard library, so you shouldn't need to install any additional modules to run this notebook.

In [10]:
import json
import csv
import urllib.request
import os

We will manually set a few items based on the date the report is being run, the date of the previous snapshot we want to compare to, and the location of the file directory.

In [11]:
## Set the date download of the older and newer jsons (yyyymmdd)
PreviousActionDate = '20171106'
ActionDate = '20200129'

#creates a location to put the results of this script
#if not os.path.exists('DCAT'):
#    os.makedirs('DCAT')

## name of the main folders containing input files and output locations
directory = 'DCATjsons/'



Before we set up a loop to check through multiple portals at once, let's look at how the comparison will work with a single portal.  

First we need to tell the script the portal name (in this case a code used by the project to identify each portal) and URL endpoint for the portal's metadata in JSON format.  We will also tell the script where to find old snapshots of this metadata and where to put a copy of the new snapshot.

In [12]:
portalName = "05b-27053" 
url = "http://gis-hennepin.opendata.arcgis.com/data.json"

oldjson = directory + '%s_%s.json' % (portalName, PreviousActionDate)
newjson = directory + '%s_%s.json' % (portalName, ActionDate)

Next we will open the webpage, load the metadata as a variable that can be immediately used by the script, and save a snapshot copy of the metadata to be used as a comparison file in the future

In [13]:
response = urllib.request.urlopen(url)
newdata = json.load(response)

with open(newjson, 'w') as outfile:  
    json.dump(newdata, outfile)

We will also check to see whether there is a snapshot of the metadata from the previous date that we can use to see how things have changed. 

In [14]:
if os.path.exists(oldjson):
    with open(oldjson) as data_file:    
        olderdata = json.load(data_file)
    print ("Comparison file found!")
else:
    print ("There is no comparison file!")

Comparison file found!


If you open the metadata URL in a browser (and have a human-friendly json viewer plugin in your browser), you can look at the structure of the metadata available for each portal and resource.  

There are a few top-level elements including "dataset:" which is a list of dictionaries.  Each dictionary represents one data layer with unique metadata element (key) / metadata (value) information. 

{
@type: "dcat:Dataset",
identifier: "http://gis-hennepin.opendata.arcgis.com/datasets/c5f67030adce4b3987782a88e0652093_9",
title: "US Congressional Districts",
...
...
...
},

We are interested in printing a report that only includes information about the items that have been newly added since the last time the script was run (or in this case Nov 6th, 2017).  To do this, we first need to make a list of the identifiers for all of the items that were in the older metadata snapshot. 

In [15]:
older_ids = {}
for x in range(len(olderdata["dataset"])):
    older_ids[x] = olderdata["dataset"][x]["identifier"]
    
print (len(older_ids))

40


The next code snippet is the heart of the comparison. 
We compare the identifier of each item found in the newly downloaded metadata against the list of identifiers from the older metadata snapshot.  If it an item is NOT there, the script adds the values for them to a dictionary.  

In [16]:
newItemDict = {}

for y in range(len(newdata["dataset"])):
    identifier = newdata["dataset"][y]["identifier"]
    if identifier not in older_ids.values():
        metadata = []
        metadata.append(portalName)
        metadata.append(identifier)
        metadata.append(newdata["dataset"][y]['title'])
        metadata.append(newdata["dataset"][y]['description'])
        metadata.append(newdata["dataset"][y]['issued'])
        metadata.append(newdata["dataset"][y]['landingPage'])
        newItemDict[identifier] = metadata

print(len(newItemDict))

7


Now that we have a dictionary with information about each of the new items we can print it to a CSV report!

NewItemDict is a dictionary with item identifiers as keys and a list of metadata as values.

{
identifier1: [portal name, identifier, title, description, issued date, landing page],
identifier2: [portal name, identifier, title, description, issued date, landing page],
identifier3: [portal name, identifier, title, description, issued date, landing page],
...
}

First we need to set the field names and the order they should be printed to the spreadsheet (matching the order of metadata elements we collected for each item).  Also the path where the CSV report should be created.

In [17]:
fields = ["portalName", "identifier", "title", "description", "dateIssued", "landingPage"]

report = "newItems_simplified_%s.csv" %  (ActionDate)

with open(report, 'w', newline='') as outfile:
    csvout = csv.writer(outfile)
    csvout.writerow(fields)
    for keys in newItemDict:
        allvalues = newItemDict[keys]
        csvout.writerow(allvalues)

Check your directory for the new file!

The next code block shows what it would look like to run this script to check the contents of multiple portals.

In [18]:
newItemDict = {}

with open('PortalList.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        portalName = row['portalName']
        url = row['URL']
        print (portalName, url)
        
        oldjson = directory + '%s_%s.json' % (portalName, PreviousActionDate)
        newjson = directory + '%s_%s.json' % (portalName, ActionDate)
        
        response = urllib.request.urlopen(url)
        newdata = json.load(response)

        with open(newjson, 'w') as outfile:  
            json.dump(newdata, outfile)
            
        if os.path.exists(oldjson):
            with open(oldjson) as data_file:    
                olderdata = json.load(data_file)
        else:
            print ("There is no comparison file!")
            
        older_ids = {}
        for x in range(len(olderdata["dataset"])):
            older_ids[x] = olderdata["dataset"][x]["identifier"]

        for y in range(len(newdata["dataset"])):
            identifier = newdata["dataset"][y]["identifier"]
            if identifier not in older_ids.values():
                metadata = []
                metadata.append(portalName)
                metadata.append(identifier)
                metadata.append(newdata["dataset"][y]['title'])
                metadata.append(newdata["dataset"][y]['description'])
                metadata.append(newdata["dataset"][y]['issued'])
                metadata.append(newdata["dataset"][y]['landingPage'])
                newItemDict[identifier] = metadata

with open(report, 'w', newline='') as outfile:
    csvout = csv.writer(outfile)
    csvout.writerow(fields)
    for keys in newItemDict:
        allvalues = newItemDict[keys]
        csvout.writerow(allvalues)

05b-27053 http://gis-hennepin.opendata.arcgis.com/data.json
05b-27143 http://stlouiscountymndata-slcgis.opendata.arcgis.com/data.json
05b-27163 https://data-wcmn.opendata.arcgis.com/data.json
06a-01 http://gis-michigan.opendata.arcgis.com/data.json


The more complicated version of this script:
1) collects additional metadata elements for each item and adds blank spaces for elements that cannot be automated
2) creates reports about deleted items and the overall number of records/changes to portals
3) cleans html tags and other formatting issues out of the description
4) has more error catching and reporting